In [3]:
LIB_DIR = "../lib"

In [15]:
import os
import sys

# If notebook lives in notebooks/, go up one level and into lib/
BASE_DIR = os.path.dirname(os.getcwd())  # adjust if needed
LIB_DIR = "../lib"

if LIB_DIR not in sys.path:
    sys.path.insert(0, LIB_DIR)

from preprocessing import (
    clean_name,
    soundex_code,
    metaphone_code,
    clean_street,
    clean_city,
    clean_state,
    normalize_zip5,
    is_valid_npi,
)

print("Using preprocessing.py from:", LIB_DIR)


Using preprocessing.py from: ../lib


In [6]:


# Quick check — this should print True
print("Spot check:", is_valid_npi(1053656744))


Spot check: True


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
medicare_path = "../data/MUP_PHY_R25_P05_V20_D23_Prov_Svc.csv"
open_payments_path = "../data/OP_DTL_GNRL_PGYR2023_P01232026_01102026.csv"
pecos_path = "../data/Medicare_FFS_Public_Provider_Enrollment_Q3_2025.csv"

In [10]:
pecos = pd.read_csv(pecos_path, low_memory=False, encoding='latin')
print(f"  PECOS: {len(pecos):,} rows, {len(pecos.columns)} columns")
print(pecos.columns)

  PECOS: 2,936,748 rows, 11 columns
Index(['NPI', 'MULTIPLE_NPI_FLAG', 'PECOS_ASCT_CNTL_ID', 'ENRLMT_ID',
       'PROVIDER_TYPE_CD', 'PROVIDER_TYPE_DESC', 'STATE_CD', 'FIRST_NAME',
       'MDL_NAME', 'LAST_NAME', 'ORG_NAME'],
      dtype='object')


In [11]:
medicare = pd.read_csv(medicare_path, low_memory=False)
print(f"  Medicare: {len(medicare):,} rows, {len(medicare.columns)} columns")
print(medicare.columns)

  Medicare: 9,660,647 rows, 28 columns
Index(['Rndrng_NPI', 'Rndrng_Prvdr_Last_Org_Name', 'Rndrng_Prvdr_First_Name',
       'Rndrng_Prvdr_MI', 'Rndrng_Prvdr_Crdntls', 'Rndrng_Prvdr_Ent_Cd',
       'Rndrng_Prvdr_St1', 'Rndrng_Prvdr_St2', 'Rndrng_Prvdr_City',
       'Rndrng_Prvdr_State_Abrvtn', 'Rndrng_Prvdr_State_FIPS',
       'Rndrng_Prvdr_Zip5', 'Rndrng_Prvdr_RUCA', 'Rndrng_Prvdr_RUCA_Desc',
       'Rndrng_Prvdr_Cntry', 'Rndrng_Prvdr_Type',
       'Rndrng_Prvdr_Mdcr_Prtcptg_Ind', 'HCPCS_Cd', 'HCPCS_Desc',
       'HCPCS_Drug_Ind', 'Place_Of_Srvc', 'Tot_Benes', 'Tot_Srvcs',
       'Tot_Bene_Day_Srvcs', 'Avg_Sbmtd_Chrg', 'Avg_Mdcr_Alowd_Amt',
       'Avg_Mdcr_Pymt_Amt', 'Avg_Mdcr_Stdzd_Amt'],
      dtype='object')


In [12]:
use_cols = [
    "Covered_Recipient_Profile_ID",  # Physician Profile ID
    "Covered_Recipient_First_Name",
    "Covered_Recipient_Last_Name",
    "Recipient_Primary_Business_Street_Address_Line1",
    "Recipient_City",
    "Recipient_State",
    "Recipient_Zip_Code",
    "Covered_Recipient_NPI",  # may be missing
    "Total_Amount_of_Payment_USDollars",
    "Date_of_Payment",
    "Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name",
    "Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name",
    "Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State",
    "Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country",
    "Program_Year"
]


In [13]:
chunk_list = []
for chunk in pd.read_csv(open_payments_path, usecols=use_cols, chunksize=200_000):
    # Optional: filter, clean, or sample chunk here
    chunk_list.append(chunk)

# Concatenate after all chunks are collected
op = pd.concat(chunk_list, ignore_index=True)
print(f"Loaded dataset: {len(op):,} rows, {len(op.columns)} columns")




C:\Users\shubh\AppData\Local\Temp\ipykernel_21892\1327469268.py:2: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(open_payments_path, usecols=use_cols, chunksize=200_000):
C:\Users\shubh\AppData\Local\Temp\ipykernel_21892\1327469268.py:2: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(open_payments_path, usecols=use_cols, chunksize=200_000):
C:\Users\shubh\AppData\Local\Temp\ipykernel_21892\1327469268.py:2: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(open_payments_path, usecols=use_cols, chunksize=200_000):
C:\Users\shubh\AppData\Local\Temp\ipykernel_21892\1327469268.py:2: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(open_payments_path, usecols=use_cols, chunksiz

Loaded dataset: 14,700,786 rows, 15 columns


In [18]:
print("=== Dirty Data Audit — Original (Uncleaned) Datasets ===\n")

# --- Open Payments ---
print("--- Open Payments ---")
print(f"Total rows:      {len(op):,}")
print(f"NPI null:        {op['Covered_Recipient_NPI'].isna().sum():,}")
print(f"First name null: {op['Covered_Recipient_First_Name'].isna().sum():,}")
print(f"Last name null:  {op['Covered_Recipient_Last_Name'].isna().sum():,}")

# Check for columns that exist
for col in ['Recipient_State', 'RecipientState', 'Recipient_City', 'RecipientCity',
            'Recipient_Zip5', 'RecipientZipCode']:
    if col in op.columns:
        print(f"{col} null: {op[col].isna().sum():,}")

print(f"\nColumn names: {list(op.columns)}")

# --- Medicare ---
print(f"\n--- Medicare ---")
print(f"Total rows:      {len(medicare):,}")
for col in ['Rndrng_NPI', 'NPI']:
    if col in medicare.columns:
        print(f"{col} null: {medicare[col].isna().sum():,}")
for col in ['Rndrng_Prvdr_First_Name', 'Rndrng_Prvdr_Last_Org_Name',
            'Rndrng_Prvdr_State_Abrvtn', 'Rndrng_Prvdr_Zip5']:
    if col in medicare.columns:
        print(f"{col} null: {medicare[col].isna().sum():,}")

print(f"\nColumn names: {list(medicare.columns)}")

# --- PECOS ---
print(f"\n--- PECOS ---")
print(f"Total rows:      {len(pecos):,}")
for col in ['NPI', 'FIRST_NAME', 'LAST_NAME', 'STATE_CD', 'ORG_NAME']:
    if col in pecos.columns:
        print(f"{col} null: {pecos[col].isna().sum():,}")

print(f"\nColumn names: {list(pecos.columns)}")


=== Dirty Data Audit — Original (Uncleaned) Datasets ===

--- Open Payments ---
Total rows:      14,700,786
NPI null:        44,233
First name null: 31,498
Last name null:  31,751
Recipient_State null: 617
Recipient_City null: 1

Column names: ['Covered_Recipient_Profile_ID', 'Covered_Recipient_NPI', 'Covered_Recipient_First_Name', 'Covered_Recipient_Last_Name', 'Recipient_Primary_Business_Street_Address_Line1', 'Recipient_City', 'Recipient_State', 'Recipient_Zip_Code', 'Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name', 'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name', 'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State', 'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country', 'Total_Amount_of_Payment_USDollars', 'Date_of_Payment', 'Program_Year']

--- Medicare ---
Total rows:      9,660,647
Rndrng_NPI null: 0
Rndrng_Prvdr_First_Name null: 537,099
Rndrng_Prvdr_Last_Org_Name null: 0
Rndrng_Prvdr_State_Abrvtn null: 0
Rndrng_Prvdr_Zip5 null

In [20]:
del pecos, medicare, op

In [19]:
# Isolate the 44K no-NPI records from the ORIGINAL OP data
no_npi_raw = op[op['Covered_Recipient_NPI'].isna()].copy()
print(f"No-NPI records: {len(no_npi_raw):,}\n")

# What do we have to work with?
has_first = no_npi_raw['Covered_Recipient_First_Name'].notna()
has_last  = no_npi_raw['Covered_Recipient_Last_Name'].notna()
has_state = no_npi_raw['Recipient_State'].notna()
has_zip   = no_npi_raw['Recipient_Zip_Code'].notna()

print("Available fields among no-NPI records:")
print(f"  Has first name:        {has_first.sum():,}")
print(f"  Has last name:         {has_last.sum():,}")
print(f"  Has both names:        {(has_first & has_last).sum():,}")
print(f"  Has both names + state:{(has_first & has_last & has_state).sum():,}")
print(f"  Has both names + zip:  {(has_first & has_last & has_zip).sum():,}")
print(f"  Has NOTHING (no name): {(~has_first & ~has_last).sum():,}")

# These are the ones we can attempt fuzzy matching on
matchable = no_npi_raw[has_first & has_last & has_state].copy()
print(f"\n>>> Matchable (name + state): {len(matchable):,}")
print(f">>> Unmatchable (no name):    {len(no_npi_raw) - len(matchable):,}")


No-NPI records: 44,233

Available fields among no-NPI records:
  Has first name:        12,833
  Has last name:         12,833
  Has both names:        12,833
  Has both names + state:12,809
  Has both names + zip:  12,809
  Has NOTHING (no name): 31,400

>>> Matchable (name + state): 12,809
>>> Unmatchable (no name):    31,424
